## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gambela,ET,8.2500,34.5833,89.65,19,100,5.68,overcast clouds
1,1,Berlevag,NO,70.8578,29.0864,24.12,83,100,24.94,overcast clouds
2,2,Cape Town,ZA,-33.9258,18.4232,86.11,62,0,5.01,clear sky
3,3,Kamen-Na-Obi,RU,53.7884,81.3423,-2.45,98,92,4.07,overcast clouds
4,4,Devonport,AU,-41.1667,146.3500,60.01,82,84,7.16,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"]>= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Devonport,AU,-41.1667,146.3500,60.01,82,84,7.16,broken clouds
9,9,Bredasdorp,ZA,-34.5322,20.0403,73.27,68,42,24.16,scattered clouds
12,12,Ahipara,NZ,-35.1667,173.1667,70.86,98,98,6.29,overcast clouds
13,13,Nouadhibou,MR,20.9310,-17.0347,71.58,68,0,16.11,clear sky
16,16,Port Alfred,ZA,-33.5906,26.8910,70.27,83,98,24.61,overcast clouds
17,17,Punta Arenas,CL,-53.1500,-70.9167,62.71,59,75,34.52,broken clouds
25,25,Nanning,CN,22.8167,108.3167,52.25,66,100,3.09,light rain
27,27,Ancud,CL,-41.8697,-73.8203,71.22,58,3,6.93,clear sky
30,30,Airai,TL,-8.9266,125.4092,60.57,92,82,2.06,broken clouds
33,33,Hobart,AU,-42.8794,147.3294,59.02,76,100,4.61,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                219
City                   219
Country                215
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [7]:
preferred_cities_df.count()

City_ID                215
City                   215
Country                215
Lat                    215
Lng                    215
Max Temp               215
Humidity               215
Cloudiness             215
Wind Speed             215
Current Description    215
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Devonport,AU,60.01,broken clouds,-41.1667,146.3500,
9,Bredasdorp,ZA,73.27,scattered clouds,-34.5322,20.0403,
12,Ahipara,NZ,70.86,overcast clouds,-35.1667,173.1667,
13,Nouadhibou,MR,71.58,clear sky,20.9310,-17.0347,
16,Port Alfred,ZA,70.27,overcast clouds,-33.5906,26.8910,
17,Punta Arenas,CL,62.71,broken clouds,-53.1500,-70.9167,
25,Nanning,CN,52.25,light rain,22.8167,108.3167,
27,Ancud,CL,71.22,clear sky,-41.8697,-73.8203,
30,Airai,TL,60.57,broken clouds,-8.9266,125.4092,
33,Hobart,AU,59.02,overcast clouds,-42.8794,147.3294,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f'{lat},{lng}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        hotel_df.loc[index,"Hotel Name"] = np.nan
        print("Hotel not found... skipping.")

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
689,Jodhpur,IN,63.30,clear sky,26.2867,73.0300,"Umaid Bhawan Palace, Jodhpur"
691,Isla Vista,US,67.84,clear sky,34.4133,-119.8610,Best Western Plus South Coast Inn
699,Mountain View,US,59.56,haze,37.3861,-122.0838,Residence Inn by Marriott Palo Alto Mountain View
701,Celestun,MX,74.34,overcast clouds,20.8667,-90.4000,Posada Lilia
702,Tome,CL,72.21,clear sky,-36.6167,-72.9500,Hotel Saint Denis
712,Yerraguntla,IN,74.21,broken clouds,14.6372,78.5395,M Y R Lodge
713,Serenje,ZM,61.68,overcast clouds,-13.2325,30.2352,The Kabaso's
714,Praia Da Vitoria,PT,61.75,broken clouds,38.7333,-27.0667,Hotel Branco I
716,Spring Hill,US,64.90,overcast clouds,28.4769,-82.5255,Room of The world By traveline Plus
717,Nalut,LY,61.61,broken clouds,30.3333,10.8500,NaN


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   198
Country                198
Max Temp               198
Current Description    198
Lat                    198
Lng                    198
Hotel Name             198
dtype: int64

In [12]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Devonport,AU,60.01,broken clouds,-41.1667,146.3500,Gateway Hotel by Nightcap Plus
9,Bredasdorp,ZA,73.27,scattered clouds,-34.5322,20.0403,Bredasdorp Country Manor
12,Ahipara,NZ,70.86,overcast clouds,-35.1667,173.1667,Ahipara Holiday Park
13,Nouadhibou,MR,71.58,clear sky,20.9310,-17.0347,El Medina
16,Port Alfred,ZA,70.27,overcast clouds,-33.5906,26.8910,The Halyards Hotel
17,Punta Arenas,CL,62.71,broken clouds,-53.1500,-70.9167,Hotel Hain
25,Nanning,CN,52.25,light rain,22.8167,108.3167,Nanning Travelling With Hostel
27,Ancud,CL,71.22,clear sky,-41.8697,-73.8203,Hotel Arena Gruesa
33,Hobart,AU,59.02,overcast clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
34,Ushuaia,AR,64.02,scattered clouds,-54.8000,-68.3000,Albatross Hotel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = ("WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level = 1.5)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# 11b. Display the figure
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))